In [35]:
import numpy as np
# numpy 출력 옵션 설정: 배열의 전체 값을 출력
np.set_printoptions(threshold=np.inf)

import pandas as pd
df_hdong = pd.read_csv('행정동명_말소제외.csv')
df_hdong = df_hdong.loc[:,'행정동코드':'읍면동명']
df_hdong['지역명'] = np.where(
    df_hdong['시도명'] == '세종특별자치시',  # '시도명'이 '세종특별자치시'인 경우
    df_hdong['시도명'] + ' ' + df_hdong['읍면동명'],  # '시도명 + 읍면동명'만 사용 
             df_hdong['시도명'] + ' ' + df_hdong['시군구명'] + ' ' + df_hdong['읍면동명'] )

#print(df_hdong['지역명'].unique())

emd_oringin = df_hdong['지역명'].unique()
len(emd_oringin)

3620

In [36]:
def group_similar_districts(districts):
    import re
    from collections import defaultdict

    # 딕셔너리를 사용하여 유사한 동을 그룹화합니다.
    district_dict = defaultdict(list)
    for district in districts:
        # 동 이름에서 숫자나 알파벳으로 끝나는 부분을 제거하여 기본 이름을 추출합니다.
        base_name = re.sub(r'(\s*\d+동|\s*[a-zA-Z]+동)$', '동', district)
        district_dict[base_name].append(district)

    # 그룹화된 동 이름 리스트를 생성합니다.
    grouped_districts = []
    for base_name, similar_districts in district_dict.items():
        if len(similar_districts) > 1:
            # 유사한 동이 여러 개 있으면 하나로 묶어서 추가합니다.
            grouped_districts.append(base_name)
        else:
            # 유사한 동이 없으면 원래 이름을 추가합니다.
            grouped_districts.append(similar_districts[0])

    return grouped_districts

grouped_list = group_similar_districts(emd_oringin)

In [54]:
#print(len(grouped_list))

#[:737] - 방호진
#[737:1473] - 황유경
#[1474: 2210]- 김무연
#[2210:] - 신예림
#print(1473+736)

2209


In [68]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep


# 크롬 옵션 설정
options = webdriver.ChromeOptions()
# 본인 환경에 맞는 user-agent 설정 필요 - 'user-agent 확인 웹사이트' 구글링
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36')
# 본인 디스플레이 해상도 확인
options.add_argument('window-size=1920,1080')
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 크롬 드라이버 설정 (
service = Service(executable_path='chromedriver')  # chromedriver의 경로를 지정 필요
driver = webdriver.Chrome(options=options)

# 대기 시간 설정
driver.implicitly_wait(5)

# 데이터를 저장할 리스트
store_data = []

# 검색할 지역 리스트 (예시)
emd = grouped_list
#emd = ['서울특별시 종로구 청운효자동', '서울특별시 종로구 사직동']


# 가게 정보를 크롤링하는 함수
def get_store_review(region, keyword):
    try:
        # 네이버 지도 검색 페이지 열기
        search_keyword = f'{region} {keyword}'
        URL = f'https://map.naver.com/v5/search/{search_keyword}'
        driver.get(URL)
        sleep(2)  # 페이지 로딩 대기

        # 검색 결과 iframe으로 전환
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'searchIframe')))
            driver.switch_to.frame('searchIframe')
            print("검색 결과 iframe으로 전환했습니다.")
        except Exception as e:
            print(f"검색 결과 iframe으로 전환 실패: {e}")
            return

        page = 1
        while True:
            # 가게 리스트 수집
            try:
                stores = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
                ))
            except TimeoutException:
                print(f"{search_keyword}: 검색 결과가 없습니다.")
                break

            print(f"{search_keyword}: 현재 {page} 페이지 / 총 {len(stores)}개의 가게를 찾았습니다.\n")

            for index, store in enumerate(stores, start=1):
                try:
                    # 가게 이름 요소 가져오기
                    store_name_element = store.find_element(By.CSS_SELECTOR, 'div.CHC5F > a.tzwk0 > div > div > span.place_bluelink.TYaxT')
                    store_name = store_name_element.text
                    print(f"{index}. {store_name}")

                    # 가게 상세 페이지로 이동
                    driver.execute_script("arguments[0].click();", store_name_element)
                    sleep(2)

                    # 상세 정보 iframe으로 전환
                    driver.switch_to.default_content()
                    try:
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'entryIframe')))
                        driver.switch_to.frame('entryIframe')
                        print("상세 정보 iframe으로 전환했습니다.")
                    except Exception as e:
                        print(f"상세 정보 iframe으로 전환 실패: {e}")
                        continue

                    # 가게 정보 수집
                    try:
                        # 가게 이름
                        store_name = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.GHAhO').text
                    except Exception as e:
                        print(f"가게 이름 추출 실패: {e}")
                        store_name = '정보 없음'

                    try:
                        # 카테고리
                        category = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.lnJFt').text
                    except Exception as e:
                        print(f"카테고리 추출 실패: {e}")
                        category = '정보 없음'

                    try:
                        # 방문자 리뷰 수
                        visited_review = driver.find_element(By.XPATH, "//span[@class='PXMot']//a[contains(text(), '방문자 리뷰')]").text
                    except Exception as e:
                        print(f"방문자 리뷰 수 추출 실패: {e}")
                        visited_review = '정보 없음'

                    try:
                        # 블로그 리뷰 수
                        blog_review = driver.find_element(By.XPATH, "//span[@class='PXMot']//a[contains(text(), '블로그 리뷰')]").text
                    except Exception as e:
                        print(f"블로그 리뷰 수 추출 실패: {e}")
                        blog_review = '정보 없음'

                    try:
                        # 평점
                        rating = driver.find_element(By.CSS_SELECTOR, 'span.PXMot.LXIwF').text
                    except Exception as e:
                        print(f"평점 추출 실패: {e}")
                        rating = '정보 없음'

                    try:
                        # 주소
                        address = driver.find_element(By.CSS_SELECTOR, 'span.LDgIH').text
                    except Exception as e:
                        print(f"주소 추출 실패: {e}")
                        address = '정보 없음'

                    try:
                        # 전화번호
                        phone_num = driver.find_element(By.CSS_SELECTOR, 'span.xlx7Q').text
                    except Exception as e:
                        print(f"전화번호 추출 실패: {e}")
                        phone_num = '정보 없음'
                    '''
                    try:
                        # 혼밥 좋아요 수
                        hb_like_num = driver.find_element(By.XPATH, "//span[@class='t3JSf' and contains(text(), '\"혼밥하기 좋아요\"')]/following-sibling::span[@class='CUoLy']/text()[normalize-space()]").text
                    except Exception as e:
                        print(f"혼밥 좋아요 수 추출 실패: {e}")
                        hb_like_num = '정보 없음'
                    '''
                 
                    # 수집한 데이터를 리스트에 저장
                    store_data.append({
                        '지역명': region,
                        '가게 이름': store_name,
                        '카테고리': category,
                        '평점': rating,
                        '방문자 리뷰 수': visited_review,
                        '블로그 리뷰 수': blog_review,
                        #'혼밥 좋아요 수': hb_like_num,
                        '주소': address,
                        '전화번호': phone_num
                    })
                    print(f"수집된 데이터: {store_data[-1]}")

                    # 다시 검색 결과 iframe으로 전환
                    driver.switch_to.default_content()
                    driver.switch_to.frame('searchIframe')
                except Exception as e:
                    print(f"{index}번 가게 정보 수집 중 오류 발생: {e}")
                    # 오류 발생 시 상세 페이지에서 검색 결과 페이지로 돌아가기
                    driver.switch_to.default_content()
                    driver.switch_to.frame('searchIframe')
                    continue

            # 다음 페이지로 이동
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'a.btn_next')
                aria_disabled = next_button.get_attribute('aria-disabled')
                if aria_disabled == 'false':
                    driver.execute_script("arguments[0].click();", next_button)
                    sleep(2)
                    page += 1
                else:
                    print(f"{search_keyword}: 마지막 페이지입니다.")
                    break
            except NoSuchElementException:
                print(f"{search_keyword}: 다음 페이지 버튼을 찾을 수 없습니다.")
                break

    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
    finally:
        # 프레임에서 빠져나오기
        driver.switch_to.default_content()

# 검색어 설정
keyword = '혼밥 맛집'  # 필요한 검색어로 변경

# 각 지역에 대해 크롤링 수행
for region in emd[437:737]:
    get_store_review(region, keyword)

# 크롤링 완료 후 브라우저 닫기
driver.quit()

# Pandas DataFrame으로 변환
df = pd.DataFrame(store_data)



검색 결과 iframe으로 전환했습니다.
대구광역시 달성군 다사읍서재출장소 혼밥 맛집: 검색 결과가 없습니다.
검색 결과 iframe으로 전환했습니다.
대구광역시 달성군 유가읍 혼밥 맛집: 현재 1 페이지 / 총 10개의 가게를 찾았습니다.

1. 백소정 대구현풍테크노폴리스점
상세 정보 iframe으로 전환했습니다.
평점 추출 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.PXMot.LXIwF"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A0469412+29090]
	(No symbol) [0x00007FF6A03DE239]
	(No symbol) [0x00007FF6A029B1DA]
	(No symbol) [0x00007FF6A02EEFE7]
	(No symbol) [0x00007FF6A02EF23C]
	(No symbol) [0x00007FF6A03397C7]
	(No symbol) [0x00007FF6A031672F]
	(No symbol) [0x00007FF6A03365A2]
	(No symbol) [0x00007FF6A0316493]
	(No symbol) [0x00007FF6A02E09D1]
	(No symbol) [0x00007FF6A02E1B31]
	GetHandleVerifier [0x00007FF6A078871D+3302573]
	GetHandleVerifier [0x00007FF6A07D4243+3612627]
	GetHandleVerifier [0x00

In [63]:
#df.to_csv('식당_크롤링_0_to_137.csv',index=False)

In [69]:
df
#df.to_csv('식당_크롤링_137_to_237.csv',index=False)

,지역명,가게 이름,카테고리,평점,방문자 리뷰 수,블로그 리뷰 수,주소,전화번호
0,대구광역시 달성군 유가읍,백소정 대구현풍테크노폴리스점,일식당,정보 없음,"방문자 리뷰 12,910",블로그 리뷰 224,"대구 달성군 유가읍 테크노중앙대로 240 1층 112, 113호",053-611-5015
1,대구광역시 달성군 유가읍,마이카츠 현풍테크노폴리스점,돈가스,별점\n4.63,방문자 리뷰 435,블로그 리뷰 67,대구 달성군 유가읍 테크노상업로 112 월드프라자 113호,053-616-1120
2,대구광역시 달성군 유가읍,한솥도시락 테크노폴리스봉리점,"도시락,컵밥",별점\n4.47,방문자 리뷰 614,블로그 리뷰 11,대구 달성군 유가읍 테크노상업로 112 1층 103호,053-611-5852
3,대구광역시 달성군 유가읍,오유미당 대구현풍테크노점,돈가스,별점\n4.8,"방문자 리뷰 2,902",블로그 리뷰 389,대구 달성군 유가읍 테크노상업로 100 1층 (유가파출소 맡은편. KT플라자옆),0507-1363-5625
4,대구광역시 달성군 유가읍,고봉민김밥인 대구테크노폴리스점,분식,별점\n4.4,방문자 리뷰 785,블로그 리뷰 12,대구 달성군 유가읍 테크노상업로 100 대구 달성군 유가면 유가면사무소 맞은편,053-611-8285
...,...,...,...,...,...,...,...,...
2445,세종특별자치시 나성동,에바돈가츠 세종점,돈가스,정보 없음,방문자 리뷰 341,블로그 리뷰 47,세종 한누리대로 245 1층 115호,0507-1346-3835
2446,세종특별자치시 나성동,김밥천국 세종점,분식,정보 없음,방문자 리뷰 659,블로그 리뷰 11,세종 한누리대로 193 1층 115호,044-864-4599
2447,세종특별자치시 나성동,애플꼬마김밥 나성점,분식,정보 없음,방문자 리뷰 745,블로그 리뷰 22,세종 나성북로 30 1층 104호,0507-1331-8686
2448,세종특별자치시 나성동,돈카츠마켙 다정점,돈가스,별점\n4.53,방문자 리뷰 227,블로그 리뷰 28,세종 한누리대로 321 SR파크시티 142호,044-868-2342


In [70]:
#df.to_csv('식당_크롤링_437_to_737.csv',index=False)

In [119]:
df1 = pd.read_csv('식당_크롤링_0_to_137.csv')
df2 = pd.read_csv('식당_크롤링_137_to_237.csv')
df3 = pd.read_csv('식당_크롤링_237_to_437.csv')
df4 = pd.read_csv('식당_크롤링_437_to_737.csv')

In [120]:
df_concat = pd.concat([df1, df2,df3,df4]).reset_index(drop=True)

In [121]:
df_concat

,지역명,가게 이름,카테고리,평점,방문자 리뷰 수,블로그 리뷰 수,주소,전화번호
0,서울특별시 종로구 청운효자동,청담동 마녀김밥 광화문점,김밥,별점\n4.36,"방문자 리뷰 4,860",블로그 리뷰 210,서울 종로구 새문안로 105,0507-1458-1146
1,서울특별시 종로구 청운효자동,KFC 광화문점,햄버거,별점\n4.3,"방문자 리뷰 6,134",블로그 리뷰 103,서울 종로구 세종대로23길 7,02-738-9870
2,서울특별시 종로구 청운효자동,진심,일식당,별점\n4.58,"방문자 리뷰 1,253","블로그 리뷰 1,010",서울 종로구 필운대로 42-1 1층,070-7789-5820
3,서울특별시 종로구 청운효자동,히타토제면소,"우동,소바",정보 없음,방문자 리뷰 771,블로그 리뷰 456,서울 종로구 옥인길 19 1층 102호,010-4141-0092
4,서울특별시 종로구 청운효자동,현대그린푸드 교보광화문점,한식,별점\n4.3,방문자 리뷰 933,정보 없음,서울 종로구 종로 1,정보 없음
...,...,...,...,...,...,...,...,...
5223,세종특별자치시 나성동,에바돈가츠 세종점,돈가스,정보 없음,방문자 리뷰 341,블로그 리뷰 47,세종 한누리대로 245 1층 115호,0507-1346-3835
5224,세종특별자치시 나성동,김밥천국 세종점,분식,정보 없음,방문자 리뷰 659,블로그 리뷰 11,세종 한누리대로 193 1층 115호,044-864-4599
5225,세종특별자치시 나성동,애플꼬마김밥 나성점,분식,정보 없음,방문자 리뷰 745,블로그 리뷰 22,세종 나성북로 30 1층 104호,0507-1331-8686
5226,세종특별자치시 나성동,돈카츠마켙 다정점,돈가스,별점\n4.53,방문자 리뷰 227,블로그 리뷰 28,세종 한누리대로 321 SR파크시티 142호,044-868-2342


In [122]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5228 entries, 0 to 5227
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   지역명       5228 non-null   object
 1   가게 이름     5228 non-null   object
 2   카테고리      5228 non-null   object
 3   평점        5228 non-null   object
 4   방문자 리뷰 수  5228 non-null   object
 5   블로그 리뷰 수  5228 non-null   object
 6   주소        5228 non-null   object
 7   전화번호      5228 non-null   object
dtypes: object(8)
memory usage: 326.9+ KB


In [123]:
#df_concat.to_csv('혼밥식당_크롤링_0_to_737.csv', index=False)

In [124]:
# 데이터타입 변경을 위한 전처리
df_concat['평점'] = df_concat['평점'].apply(lambda x: x.replace('별점\n',''))
df_concat['방문자 리뷰 수'] = df_concat['방문자 리뷰 수'].apply(lambda x: x.replace('방문자 리뷰',''))
df_concat['방문자 리뷰 수'] = df_concat['방문자 리뷰 수'].apply(lambda x: x.replace(',',''))

df_concat['블로그 리뷰 수'] = df_concat['블로그 리뷰 수'].apply(lambda x: x.replace('블로그 리뷰',''))
df_concat['블로그 리뷰 수'] = df_concat['블로그 리뷰 수'].apply(lambda x: x.replace(',',''))
df_concat = df_concat[(df_concat['평점'] != '정보 없음') & (df_concat['방문자 리뷰 수'] != '정보 없음') & (df_concat['블로그 리뷰 수'] != '정보 없음')]


In [125]:
df_concat.reset_index(inplace=True, drop=True)
df_concat

,지역명,가게 이름,카테고리,평점,방문자 리뷰 수,블로그 리뷰 수,주소,전화번호
0,서울특별시 종로구 청운효자동,청담동 마녀김밥 광화문점,김밥,4.36,4860,210,서울 종로구 새문안로 105,0507-1458-1146
1,서울특별시 종로구 청운효자동,KFC 광화문점,햄버거,4.3,6134,103,서울 종로구 세종대로23길 7,02-738-9870
2,서울특별시 종로구 청운효자동,진심,일식당,4.58,1253,1010,서울 종로구 필운대로 42-1 1층,070-7789-5820
3,서울특별시 종로구 청운효자동,밥+,한식,4.42,547,438,서울 종로구 옥인길 9,0507-1391-1293
4,서울특별시 종로구 청운효자동,공기식당,카레,4.48,561,673,서울 종로구 필운대로6길 20-1 1층,0507-1440-0930
...,...,...,...,...,...,...,...,...
3607,세종특별자치시 나성동,프랭크버거 세종나성점,햄버거,4.76,585,43,세종 한누리대로 292 행복의아침 106호,044-867-8485
3608,세종특별자치시 나성동,덴까이,일본식라면,4.63,453,266,세종 한누리대로 211 109호 덴까이,070-7630-8111
3609,세종특별자치시 나성동,24시전주명가콩나물국밥 세종나성점,국밥,4.42,1921,105,"세종 한누리대로 287 1층108,109호(나성동 세종포레뷰1차)",0507-1376-1666
3610,세종특별자치시 나성동,돈카츠마켙 다정점,돈가스,4.53,227,28,세종 한누리대로 321 SR파크시티 142호,044-868-2342


In [127]:
# 데이터 타입 변경
df_concat['평점'] = df_concat['평점'].astype('float64') 
df_concat['방문자 리뷰 수'] = df_concat['방문자 리뷰 수'].astype('int64') 
df_concat['블로그 리뷰 수'] = df_concat['블로그 리뷰 수'].astype('int64')

C:\Users\samsung-user\AppData\Local\Temp\ipykernel_31348\1706169095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['평점'] = df_concat['평점'].astype('float64')
C:\Users\samsung-user\AppData\Local\Temp\ipykernel_31348\1706169095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['방문자 리뷰 수'] = df_concat['방문자 리뷰 수'].astype('int64')
C:\Users\samsung-user\AppData\Local\Temp\ipykernel_31348\1706169095.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [128]:
df_concat

,지역명,가게 이름,카테고리,평점,방문자 리뷰 수,블로그 리뷰 수,주소,전화번호
0,서울특별시 종로구 청운효자동,청담동 마녀김밥 광화문점,김밥,4.36,4860,210,서울 종로구 새문안로 105,0507-1458-1146
1,서울특별시 종로구 청운효자동,KFC 광화문점,햄버거,4.30,6134,103,서울 종로구 세종대로23길 7,02-738-9870
2,서울특별시 종로구 청운효자동,진심,일식당,4.58,1253,1010,서울 종로구 필운대로 42-1 1층,070-7789-5820
3,서울특별시 종로구 청운효자동,밥+,한식,4.42,547,438,서울 종로구 옥인길 9,0507-1391-1293
4,서울특별시 종로구 청운효자동,공기식당,카레,4.48,561,673,서울 종로구 필운대로6길 20-1 1층,0507-1440-0930
...,...,...,...,...,...,...,...,...
3607,세종특별자치시 나성동,프랭크버거 세종나성점,햄버거,4.76,585,43,세종 한누리대로 292 행복의아침 106호,044-867-8485
3608,세종특별자치시 나성동,덴까이,일본식라면,4.63,453,266,세종 한누리대로 211 109호 덴까이,070-7630-8111
3609,세종특별자치시 나성동,24시전주명가콩나물국밥 세종나성점,국밥,4.42,1921,105,"세종 한누리대로 287 1층108,109호(나성동 세종포레뷰1차)",0507-1376-1666
3610,세종특별자치시 나성동,돈카츠마켙 다정점,돈가스,4.53,227,28,세종 한누리대로 321 SR파크시티 142호,044-868-2342
